In [2]:
%reload_ext autoreload
%autoreload 1
%aimport train

In [21]:
import pandas as pd
import numpy as np

In [78]:
num_docs = 10000

tweet_df = pd.read_pickle('tweet_data_tokenized.pkl')
tweet_df = tweet_df.sort_values(by='id', ascending=False)
tweet_df = tweet_df.head(num_docs)
tweet_df = tweet_df.reset_index()

In [94]:
import pickle

with open('tweet_vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)

In [79]:
def build_context(v, context_size=3):
    k = context_size
    v = np.lib.pad(v, (k,0), 'constant', constant_values=(-1))
    for i in range(len(v) - k):
        yield v[i:i+k],v[i+k]

data = [(tweet_id, context, target) for tweet_id, words in tweet_df.tokenized.iteritems() for context, target in build_context(words)]

docs     = np.array([x[0] for x in data])
contexts = np.array([x[1] for x in data])
targets  = np.array([x[2] for x in data])

In [84]:
import tensorflow as tf
import tflearn

In [131]:
g = tf.Graph()

In [132]:
num_words = len(vocab)

with g.as_default():
    input_layer = tflearn.input_data(shape=[None, 1])
    embedding_layer, = tflearn.embedding(input_layer, input_dim=num_docs, output_dim=100)
    softmax = tflearn.fully_connected(embedding_layer, num_words, activation='softmax')

    sgd = tflearn.SGD(learning_rate=0.1, lr_decay=0.96, decay_step=1000)
    top_k = tflearn.metrics.Top_k(3)
    net = tflearn.regression(softmax, optimizer=sgd, metric=top_k,
                             loss='categorical_crossentropy')

In [133]:
with g.as_default():
    model = tflearn.DNN(net, tensorboard_verbose=0)

In [ ]:
with g.as_default():
    X = docs[:,np.newaxis]
    Y = tflearn.data_utils.to_categorical(targets, num_words)
    model.fit(X, Y, n_epoch=20, show_metric=True, run_id="dense_model")

Training Step: 3890  | total loss: 6.55533
| SGD | epoch: 001 | loss: 6.55533 - top3: 0.2909 -- iter: 083008/165934
